In [1]:
import Pkg

In [2]:
using Graphs
using GraphPlot


###################################################
## Graph: IN/OUT
###################################################

function Read_undirected_Graph_DIMACS(filename)

    g=SimpleGraph(1)	 # Creation of an undirected graph with 1 node

    open(filename) do f
       
           for (i,line) in enumerate(eachline(f))
              x = split(line," ") # For each line of the file, splitted using space as separator
              if(x[1]=="p")       # A line beginning with a 'p' gives the graph size
                  n = parse(Int,x[3])
                  g = SimpleGraph(n)  # Recreation of a undirected graph with n nodes
              elseif(x[1] == "e") # A line beginning with a 'e' gives the edges
                  v_1 = parse(Int, x[2])
                  v_2 = parse(Int, x[3])
                  add_edge!(g,v_1,v_2)
                  #g.weights[v_1,v_2] = 1  # without edge weight
               end
           end
    end
    return g
end


function Read_directed_Graph_GRA(filename)

    g=SimpleDiGraph(1)	 # Creation of a directed graph with 1 node

    open(filename) do f
       
           for (i,line) in enumerate(eachline(f))
              x = split(line," ") # For each line of the file, splitted using space as separator
              if (i==1)
                  n = parse(Int,x[1])  # Read the number of nodes
                  g = SimpleDiGraph(n)  # Recreation of a directed graph with n nodes
              else
                  j=3
                  while (j<size(x)[1]-1)
                    v_1 = parse(Int, x[1])
                    v_2 = parse(Int, x[j])
                    add_edge!(g,v_1+1,v_2+1)   # in this case an edge is oriented as an arc
                    #g.weights[v_1+1,v_2+] = 1  # without arc weight
                    j=j+1
                  end
              end
           end
    end
    return g
end




function WritePdf_visualization_Graph(G,filename)

filename_splitted_in_two_parts = split(filename,".") # split to remove the file extension
filename_with_pdf_as_extension= filename_splitted_in_two_parts[1]*".pdf"
# save to pdf
draw(PDF(filename_with_pdf_as_extension, 16cm, 16cm), gplot(G, nodelabel = 1:nv(G)))
end


WritePdf_visualization_Graph (generic function with 1 method)

In [3]:
using JuMP
using CPLEX
using CPUTime

include("Graph_StableSet.jl")

function PLNE_compact_StableSet(G)

   m = Model(CPLEX.Optimizer)

   @variable(m, x[1:nv(G)], Bin)

   @objective(m, Max, sum(x[i] for i = 1:nv(G) ) )

   for e in edges(G)
       @constraint(m, x[src(e)] + x[dst(e)] <= 1 )
   end

   print(m)
   println()

   optimize!(m)
   
   println(solution_summary(m, verbose=true))

   status = termination_status(m)

   if status == JuMP.MathOptInterface.OPTIMAL
       println("Valeur optimale = ", objective_value(m))
       println("Solution primale optimale :")
      S = Bool[]
       for i= 1:nv(G)
         println("\t x[",i,"] = ", value(x[i]))
         if (value(x[i])<0001) 
           push!(S,0)
         else
           push!(S,1)
         end
       end
       println("Temps de résolution :", solve_time(m))
       return S
   else
      println("Problème lors de la résolution")
   end

end

##########################
### Launch compact PLNE and visualization for the stable set problem
##########################

function Find_exact_StableSet(filename)

 G=Read_undirected_Graph_DIMACS(filename)

 @time @CPUtime S=PLNE_compact_StableSet(G)

 WritePdf_visualisation_Stable(G,S,filename)

end


Find_exact_StableSet (generic function with 1 method)

In [4]:
Find_exact_StableSet("Instances/DIMACS/0023_myciel4.dim")


Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Found incumbent of value 0.000000 after 0.01 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 71 rows, 23 columns, and 142 nonzeros.
Reduced MIP has 23 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.09 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 71 rows, 23 columns, and 142 nonzeros.
Reduced MIP has 23 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.13 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 71.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.13 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                            0.0000   